# Concat tout les jours tempo  
Ce notebook concat tout les jours tempo en un seul fichier

In [ ]:
import pandas as pd
import os

In [ ]:
in_relative_path = "../raw_datasets/tempo/"
in_absolute_path = os.path.abspath(os.path.join(os.getcwd(), in_relative_path))

all_tempo_path = []
for y in range(2014,2024):
    all_tempo_path.append(in_absolute_path + f"/eCO2mix_RTE_tempo_{y}-{y+1}.xlsx")
all_tempo_path

In [ ]:
tempo_df = pd.DataFrame()
for path in all_tempo_path:
    temporary_df = pd.read_excel(path)
    temporary_df = temporary_df.drop(temporary_df.tail(1).index)
    tempo_df = pd.concat([tempo_df, temporary_df])

tempo_df.reset_index()

In [ ]:
out_relative_path = "../clean_datasets/"
out_absolute_path = os.path.abspath(os.path.join(os.getcwd(), out_relative_path))

tempo_df.to_csv(f"{out_absolute_path}/tempo_2014_2024.csv", index=False)